## Bayesian inference analysis notebook

This is an example notebook about how to use our tools to analysis a observation/nuclear constraint on neutron star equation of state. 

Here in this notebook, we are using a RMF EoS model (in the future we will implement more equation of state)

In [9]:
#Package we need:
import InferenceWorkflow.BayesianSampler as sampler
import InferenceWorkflow.Likelihood as likelihood
import InferenceWorkflow.prior as prior

import math
import numpy as np

from TOVsolver.constant import oneoverfm_MeV, m_rho, m_w,G,c
import TOVsolver.main as main
import EOSgenerators.crust_EOS as crust
import EOSgenerators.RMF_EOS as RMF

We need to define the crust, this crust and interface is unversial for all the equation of state here, we just change the core part of equation of state and that is come from the RMF computation. Crust as below, "Tolos_crust_out.txt" is BPS crust model. Then as in README, since that file energy_density and presurssure all in MeV/fm3, so we need some conversion. then just call 

   ```sh
   crust.PolyInterpolate
   ```

To finish the interface and make it ready to connect with core part

In [10]:
Tolos_crust_out = np.loadtxt('Tolos_crust_out.txt', delimiter='  ')
eps_crust_out = Tolos_crust_out[:,3] * G / c**2
pres_crust_out = Tolos_crust_out[:,4] * G / c**4

eps_crust, pres_crust = crust.PolyInterpolate(eps_crust_out, pres_crust_out)


### Set up prior

Next step, we need to set up the prior, first use parameters array to specify the variable name, should consistent with what you need to call them.

Define a prior transform function to define prior. Cube are set of random number from 0 to 1. This prior setting is standard set-up of UltraNest package, since we are using UltraNest to do nest-sampling. We provided 

"normal_Prior" and "flat_prior"

two options call from prior. Here then the Parameters prior should all set

------------------

However, since we are doing Equation of state Inference from mass radius of neutron star measurement. The center density of the star should be also sampled. Otherwise will be a partially-defined prior, did not span all parameters space, and proved to be different with full-scope inference.

This request as randomly generate a density from a EoS range, however, this process is not that trivial, since we need to determine the upper limit of the central density of neutron star ---  different equation of state will predict different upper bound, so here we need to use the prior-setting EoS parameters computing the EOS by

```sh
RMF.compute_EOS
```

Compute out EOS, put into

```sh
main.OutputMR
```

find out Mass Radius of this equation of state, find out the last stable point of this equation of state.(first mass points that give the direvative to be negative)

found out that index by len() function, then reset this max_d to be upper limit of this density range.

In [11]:
parameters = ['g_sigma', 'g_omega','g_rho', 'kappa', 'lambda_0', 'zeta', 'Lambda_w','d1']

def prior_transform(cube):
    params = cube.copy()
    params[0] = math.sqrt(prior.normal_Prior(107.5, 7.5,cube[0]))
    params[2] = math.sqrt(prior.flat_prior(75,210,cube[2]))
    params[1] = math.sqrt(prior.flat_prior(150,210,cube[1]))
    params[3] = prior.normal_Prior(2.525/oneoverfm_MeV, 1.525/oneoverfm_MeV,cube[3])
    params[4] = prior.normal_Prior(0.0045, 0.0205,cube[4])
    params[5] = prior.flat_prior(0,0.04,cube[5])
    params[6] = prior.flat_prior(0,0.045,cube[6])
    
    g_sigma = params[0]
    g_omega = params[1]
    g_rho = params[2]

    kappa = params[3]
    lambda_0 = params[4]
    zeta = params[5]
    Lambda_w = params[6]
    m_sig = 495 / oneoverfm_MeV

    theta = np.array([m_sig, m_w, m_rho, g_sigma, g_omega, g_rho, kappa,
                         lambda_0, zeta, Lambda_w])
    
    ep, pr = RMF.compute_EOS(eps_crust, pres_crust, theta)

    eps_total = np.hstack((eps_crust,ep))
    
    pres_total = np.hstack((pres_crust,pr))
    
    RFSU2R = []
    MFSU2R = []
    density = np.logspace(14.3, 15.6, 50)
    if all(x<y for x,y in zip(eps_total[:], eps_total[1:])) and all(x<y for x, y in zip(pres_total[:], pres_total[1:])):
        MR = main.OutputMR('',eps_total,pres_total).T
        if len(MR[1]) == False: 
            params[7] = 0
        else:
   
            for i in range(len(MR[1])):
                RFSU2R.append(MR[0][i])
                MFSU2R.append(MR[1][i])   
                if i > 20 and MR[1][i] - MR[1][i-1]< 0:
                    break
    if len(MFSU2R)==False:
        params[7] = 0
    else:
        max_index = len(MFSU2R)
        max_d = np.log10(density[max_index-1])
        params[7] = 14.3 + (max_d - 14.3) * cube[7]
    return params

### Set up likelihood

We need to set up a likelihood, Using standard definition way of UltraNest, that is below.

Here the likelihood is generated from a simulated mass radius measurement, which is $M = 1.4 M_{\odot}$ and $R = 13$ km, With a 5% Mass radius measurement uncertainty, so here 

   ```sh
   likelihood.MRlikihood_Gaussian
   ```
function will be use for our likelihood, please check [likelihood.MRlikihood_Gaussian](https://github.com/ChunHuangPhy/EoS_inference/blob/main/InferenceWorkflow/Likelihood.py) to see the original code, and more choice of likelihood.
eg:
1. If we have some real mass-radius measurements, say PSR J0030 or PSR J0740, come from NICER, a KDE kernel could be trained to feed into 

   ```sh
   likelihood.MRlikihood_kernel(eps_crust,pres_crust,x,theta)
   ```
set the KDE kernel as a input for this function

2. If we gain measurement from radio-timing, say only measure the neutron star mass, then

   ```sh
   likelihood.Masslikihood_Gaussian(eps_crust,pres_crust,x,theta)
   ```
Which will give the likelihood from single mass measurement, x is the parameters of that measurement, you should specify where this measurement mass is located and what is the sigma width of this mass measurement

3. If we have nuclear measurements, and want to constrain this RMF model by nuclear properties like K(The Incompressibility of nuclear matter),J ( the symmetry energy at saturation density) and L( the slope of symmetry energy at saturation density). You can choose:

   ```sh
   likelihood.Kliklihood(theta,K_low,K_up)
   likelihood.Jliklihood(theta,K_low,K_up)
   likelihood.Lliklihood(theta,K_low,K_up)
   ```
   
We are defaulting a hard-cut flat constrain, so if you don't like this default hard cut, also could define the likelihood by youself with similiar style.

In [12]:
def likelihood_transform(theta):
    prob = likelihood.MRlikihood_Gaussian(eps_crust,pres_crust,(1.4,13,0.07,0.65),theta)
    #prob = likelihood.Kliklihood(theta,100,400)
    return prob

### Set up sampler

Here next, we define sampler, there is two different sampler we provided for. Considering where you need resume file:

```sh
sampler.UltranestSampler   and  sampler.UltranestSamplerResume
```

Here since it is our first run, so we only use first one. some of the sampler parameters is requested, first is step number, our choice for UltraNest sampler is slicesampler, which could easily be sliced up your total computation load, and parallelize, speed up sampling. So step as suggested by documentation of UltraNest, we use 2*len(parameters).

live_point we set 2000, it will influence the sampling precision, We suggest for 7 dimension space, maybe 5000 is a better choice, however, since my computer only have limited resources, we set 2000.

max_calls set 10000, it is how many iteration after it will stop, we suggest to set this number significantly higher, otherwise maybe will broken before the inference converging to a definite value. That result will be un-phyiscal


In [13]:
step = 2 * len(parameters)
live_point = 2000

max_calls = 10000
samples = sampler.UltranestSampler(parameters,likelihood_transform,prior_transform,step,live_point,max_calls)


Creating directory for new run output/run14
should have some in mr


capi_return is NULL
Call-back cb_fcn_in___user__routines failed.


[ultranest] Sampling 400 live points from prior ...


capi_return is NULL
Call-back cb_fcn_in___user__routines failed.


KeyboardInterrupt: 